In [2]:
# Install necessary libraries
!pip install -q torchaudio
!pip install -q git+https://github.com/snakers4/silero-vad
!pip install -q google-colab
!pip install -q librosa soundfile pydub
!pip install -q git+https://github.com/openai/whisper.git

# To be decided
!pip install -q googletrans==4.0.0-rc1
!pip install -q gTTS

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/drive/MyDrive/video-converter/example-video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


# Generate my way to auto process the video

In [5]:
# Path to your video file
mp4_path = '/content/drive/MyDrive/video-converter/example-video.mp4'

# Extract audio using ffmpeg
wav_path = 'audio.wav'  # Temporary WAV file for audio

!rm -f "{wav_path}"
!ffmpeg -i "{mp4_path}" -ab 160k -ac 2 -ar 44100 -vn "{wav_path}"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [6]:
from google.colab import drive
from IPython.display import HTML
from base64 import b64encode
import torchaudio
import torch

# Load the Silero VAD model
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=False)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

# Read the audio file
wav = read_audio(wav_path, sampling_rate=16000)

# Get speech timestamps
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=16000)

Using cache found in /root/.cache/torch/hub/snakers4_silero-vad_master


In [7]:
import whisper

# Load the Whisper model
model = whisper.load_model("base")

transcription = []
for i, timestamp in enumerate(speech_timestamps):
    start_time = timestamp['start']
    end_time = timestamp['end']

    # Extract the audio chunk
    start_sample = int(start_time )
    end_sample = int(end_time )
    chunk = wav[start_sample:end_sample]

    # Transcribe the audio chunk
    result = model.transcribe(chunk)

    # Print the transcription with timestamp
    transcription.append({
        "start": start_time,
        "end": end_time,
        "text": result['text']
    })
    print(f"[{start_time:.2f} - {end_time:.2f}] {result['text']}")


  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



[3616.00 - 80352.00]  If you're a beginner in the stock market, listen up because this is one of the biggest mistakes I see beginners making today.
[81952.00 - 177632.00]  So if you're a dividend investor and you see a stock like OK, and you see it has a dividend yield of 14% you'd get hell excited, right?
[180256.00 - 346592.00]  If you don't know what that means, that's like having a savings account with a 14% interest rate. So if you were to invest $1,000 in the stock, you could expect to get paid $140 each year just for owning the stock.
[350240.00 - 376288.00]  And here are some problems I see with that.
[379936.00 - 429024.00]  First and foremost, this company is in an industry that's not doing so hot right now.
[433696.00 - 475616.00]  They have 444 million shares outstanding.
[477728.00 - 810976.00]  that to pay each of those shares $3.74 every year, which gives a company a payout ratio of a whopping 187%. A healthy payout ratio is anything under 50%, but 190% means that you're

In [ ]:
from base64 import b64encode
from googletrans import Translator

translator = Translator()

for timestamp in transcription:
    start_time = timestamp['start']
    end_time = timestamp['end']

    # Translate the text to Vietnamese
    translated_text = translator.translate(timestamp["text"], dest="vi").text

    # Append to transcription with Vietnamese translation
    timestamp["vietnamese_text"] = translated_text
    print(f"[{start_time:.2f} - {end_time:.2f}] From: {timestamp['text']}\n -> To: {translated_text}")

In [38]:
%mkdir -p trans_transcript

from gtts import gTTS
import os

for timestamp in transcription:
    start_time = timestamp['start']
    end_time = timestamp['end']
    translated_text = timestamp['vietnamese_text']

    # Create a filename based on the start time
    filename = f"trans_transcript/output_{start_time:.2f}.mp3"

    # Create the speech object
    tts = gTTS(text=translated_text, lang='vi')

    # Save the speech to a file
    tts.save(filename)
    timestamp["file"] = filename
    print(f"Saved Vietnamese audio for segment [{start_time:.2f} - {end_time:.2f}] to {filename}")

Saved Vietnamese audio for segment [3616.00 - 80352.00] to trans_transcript/output_3616.00.mp3
Saved Vietnamese audio for segment [81952.00 - 177632.00] to trans_transcript/output_81952.00.mp3
Saved Vietnamese audio for segment [180256.00 - 346592.00] to trans_transcript/output_180256.00.mp3
Saved Vietnamese audio for segment [350240.00 - 376288.00] to trans_transcript/output_350240.00.mp3
Saved Vietnamese audio for segment [379936.00 - 429024.00] to trans_transcript/output_379936.00.mp3
Saved Vietnamese audio for segment [433696.00 - 475616.00] to trans_transcript/output_433696.00.mp3
Saved Vietnamese audio for segment [477728.00 - 810976.00] to trans_transcript/output_477728.00.mp3
Saved Vietnamese audio for segment [814624.00 - 887776.00] to trans_transcript/output_814624.00.mp3
Saved Vietnamese audio for segment [889376.00 - 936416.00] to trans_transcript/output_889376.00.mp3


In [39]:
import librosa
import soundfile as sf
from pydub import AudioSegment

for timestamp in transcription:
    start_time = timestamp['start']
    end_time = timestamp['end']
    filename = timestamp["file"]

    # Load the generated audio
    audio = AudioSegment.from_mp3(filename)

    # Calculate the desired duration
    desired_duration = (end_time - start_time) / 16000 * 1000  # Convert to milliseconds
    print(f"len(audio) = {len(audio)}, desired_duration = {desired_duration}")
    # Check if the audio needs to be sped up
    if len(audio) > desired_duration:
        # Calculate the speedup factor
        speed_factor = len(audio) / desired_duration
        print(f"speed_factor = {speed_factor}")
        # Use librosa to speed up the audio
        y, sr = librosa.load(filename, sr=None)
        y_stretched = librosa.effects.time_stretch(y, rate=speed_factor)

        # Save the sped-up audio
        sf.write(filename, y_stretched, sr)

        print(f"Sped up audio for segment [{start_time:.2f} - {end_time:.2f}] to match duration.")

len(audio) = 11496, desired_duration = 4796.0
speed_factor = 2.396997497914929
Sped up audio for segment [3616.00 - 80352.00] to match duration.
len(audio) = 12936, desired_duration = 5980.0
speed_factor = 2.163210702341137
Sped up audio for segment [81952.00 - 177632.00] to match duration.
len(audio) = 19176, desired_duration = 10396.0
speed_factor = 1.8445555983070412
Sped up audio for segment [180256.00 - 346592.00] to match duration.
len(audio) = 3888, desired_duration = 1628.0
speed_factor = 2.388206388206388
Sped up audio for segment [350240.00 - 376288.00] to match duration.
len(audio) = 7368, desired_duration = 3068.0
speed_factor = 2.4015645371577574
Sped up audio for segment [379936.00 - 429024.00] to match duration.
len(audio) = 4536, desired_duration = 2620.0
speed_factor = 1.7312977099236642
Sped up audio for segment [433696.00 - 475616.00] to match duration.
len(audio) = 37008, desired_duration = 20828.0
speed_factor = 1.7768388707509122
Sped up audio for segment [477728.

In [ ]:
from IPython.display import Audio

print(transcription)
for timestamp in transcription:
  print(f"[{timestamp['start']:.2f} - {timestamp['end']:.2f}] {timestamp['vietnamese_text']}")
  try:
    display(Audio(timestamp["file"]))
    pass
  except Exception as e:
    print(f"Error displaying audio for segment {timestamp['start']:.2f}: {e}")

  # Only want to check the first one
  break

In [48]:
import subprocess

# Combine audio segments into a single output file
combined_audio = AudioSegment.empty()
original_audio = AudioSegment.from_mp3(wav_path)
curr = 0
final = len(wav)
for timestamp in transcription:
    start = timestamp['start'] / 16000 * 1000
    end = timestamp['end'] / 16000 * 1000

    print(f"curr = {curr}, timestamp = [{start} : {end}]")

    if curr < start:
        # Get the curr to timestamp['start'] from original video audio
        segment_audio = original_audio[curr:start]
        combined_audio += segment_audio

    curr = end

    try:
        segment_audio = AudioSegment.from_mp3(timestamp["file"])
        combined_audio += segment_audio
    except FileNotFoundError:
        print(f"Warning: Audio file not found for segment at {timestamp['start']:.2f}. Skipping.")

# Check if we reach the final, else put that too
if curr == final:
    segment_audio = original_audio[curr:]
    combined_audio += segment_audio

# Export the combined audio
combined_audio.export("combined_audio.mp3", format="mp3")

curr = 0, timestamp = [226.0 : 5022.0]
curr = 5022.0, timestamp = [5122.0 : 11102.0]
curr = 11102.0, timestamp = [11266.0 : 21662.0]
curr = 21662.0, timestamp = [21890.0 : 23518.0]
curr = 23518.0, timestamp = [23746.0 : 26814.0]
curr = 26814.0, timestamp = [27106.0 : 29726.0]
curr = 29726.0, timestamp = [29858.0 : 50686.0]
curr = 50686.0, timestamp = [50914.0 : 55486.0]
curr = 55486.0, timestamp = [55586.0 : 58526.0]


<_io.BufferedRandom name='combined_audio.mp3'>

In [50]:
!ffmpeg -i "{mp4_path}" -i combined_audio.mp3 -c:v copy -map 0:v:0 -map 1:a:0 output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/content/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [52]:
!cp output.mp4 /content/drive/MyDrive/video-converter/output.mp4